

# OCR CODE




In [ ]:
#for installing tesseract ocr
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
#install pytesseract python package
!pip3 install pytesseract

In [ ]:
#@title Run this if you want to select card from Google drive
#Run this if you want to select visiting card from Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# -*- coding: utf-8 -*
import pytesseract
try:
    from PIL import Image
except ImportError:
    import Image
pytesseract.pytesseract.tesseract_cmd = r'tesseract' # Tesseract Location
def ocr_core(filename):
    """
    This function will handle the core OCR processing of images.
    """
    text = pytesseract.image_to_string(Image.open(filename)) #lang='eng' 
        #We'll use Pillow's Image class to open the image and pytesseract to detect the string in the image
    return text
cardPath = input("Enter file name along with path\n") #like content/john smith.jpeg
text = ocr_core('/'+cardPath)
text = [line for line in text.split("\n") if line.strip()]

In [ ]:
text #print ocr text data

# Extracting Attributes

In [ ]:
import re
#initializing all values with null values
name = "";firstName="";lastName="";companyName = "";position = "";mobileNo = "";website = "";emailId = "";linkedinId = ""
#extracting name attribute
if len(text)>=3:
    name = text[0]#name
    companyName = text[1]#company name
    position = text[2]#position in the company
else:
    name = text[0]#name
    companyName = text[1]#company name
flag2 = 0
for i in range(0,len(text)):
    #mobile no
    flag1=1
    if flag2==0:
        for element in text[i]:
            if element!=" " and element!="." and element!="-" and element not in (str(i) for i in (0,1,2,3,4,5,6,7,8,9)):
                flag1 = 0
        if text[i][0]=="+" or "phone" in text[i].lower() or "mobile" in text[i].lower() or "tel" in text[i].lower() or "cell" in text[i].lower() or (flag1==1 and flag2==0):
            mobileNo = re.sub('[^0-9]','',text[i])
            flag2 = 1
    #website
    elif ("www" in text[i] or "http" in text[i] or "https" in text[i]) and "linkedin.com/in/" not in text[i] and "@" not in text[i]:
        website = text[i]
    #email id
    elif "@" in text[i]:
        flag=0
        for m in range(len(text[i])):
            if text[i][m] == " ":
                emailId = text[i][m+1:]
                flag=1
                break
            elif ("email" in text[i].lower() or "emailid" in text[i].lower()) and text[i][m] == ":":
                emailId = text[i][m+1:]
                flag=1
                break
        if flag==0:
            emailId = text[i]
    #linkedin ID
    elif "linkedin.com/in/" in text[i]:
        linkedinId = text[i]

In [ ]:
# printing the attributes
print (name)
print (position)
print (emailId)
print (mobileNo)
print (website)
print (companyName)
print (linkedinId)

# Fetch LinkedIn data using web scraping

In [ ]:
#installing chromedriver for google chrome and selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
#chromedriver options for running chrome
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
query = name + " " + companyName
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#path to chromedriver.exe
driver = webdriver.Chrome(r"chromedriver",options=options) #chrome web driver path
driver.get('https://www.linkedin.com')
#finding sign in button
driver.find_element_by_class_name('nav__button-secondary').click()
u_email = input("Enter your LinkedIn Email")
u_pass = input("Enter Your LinkedIn Password")
driver.find_element_by_id('username').send_keys(U_email) #Enter Your Email
driver.find_element_by_id('password').send_keys(u_pass) #linkedIn password
#to press on login button
driver.find_element_by_class_name('login__form_action_container').click()
#check if verification pin is generated from linkedIn to your Email
try: #For check if security pin is generated to your mail
  findpin = driver.find_element_by_id('input__email_verification_pin')
  pin = input("Enter your pin\n") 
  findpin.send_keys(pin)
  driver.find_element_by_id('email-pin-submit-button').click()
except:
  pass
try: # For Checking if "Skip" to enter Mobile Number 
  driver.find_element_by_class_name("secondary-action").click()
except:
  pass

In [ ]:
#print the current page screen, see if what error occurs
from PIL import Image
driver.save_screenshot('/content/sr2.jpg')
img  = Image.open('/content/sr2.jpg')
img

In [ ]:
#will find the search bar & input the query
search_query = driver.find_element_by_tag_name('input')
search_query.send_keys(query)
search_query.send_keys(Keys.RETURN)

In [ ]:
driver.implicitly_wait(10)
linklist = driver.find_elements_by_xpath("//a[@class = 'search-result__result-link ember-view']") #selecting the person
linklist[0].click()
name = driver.find_element_by_xpath("//li[@class = 'inline t-24 t-black t-normal break-words']").text
#this will return the text containing current position and companyname of the person
text1 = driver.find_element_by_xpath("//h2[@class = 'mt1 t-18 t-black t-normal']").text
#for getting LinkedIn ID
contact = driver.find_element_by_link_text('Contact info')
contact.click()
linkedinId = contact.get_attribute('href')
linkedinId = linkedinId.strip("detail/contact-info/")
driver.close()
#extracting position and companyName from text1
for i in range(0,len(text1)):
    if text1[i]=='a' and text1[i+1]=='t':
        break
position = text1[0:i-1]
companyName = text1[i+3:]
#first name and last name
for i in range(0,len(name)):
    if(name[i]==" "):
        break
firstName = name[:i]
lastName = name[i+1:]

In [ ]:
#printing Updated values
print(name)
print(firstName)
print(lastName)
print(position)
print(companyName)
print(linkedinId)

# Saving into excel using drive

In [ ]:
#accessing your gmail drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#importing pandas
import pandas as pd

## Run the below part only once for creating columns and saving in drive

In [ ]:
#creating column names of table
database=pd.DataFrame(columns=['Name','FName','LName','Company','Position','Mobile','Website','Email','Linkedinid'])
database=database.astype(str)

In [ ]:
#saving the table
database.to_csv('/content/drive/My Drive/database.csv', index = False) #drive path like /content/drive/My Drive/database.csv

## Inserting Cells,No restrictions run any times

In [ ]:
#Reading the table from drive
database = pd.read_csv('/content/drive/My Drive/database.csv') #drive path like /content/drive/My Drive/database.csv

In [ ]:
#Appending the new rows to the table
database=database.append(pd.Series([name,firstName,lastName,companyName,position,mobileNo,website,emailId,linkedinId], index=database.columns ), ignore_index=True)

In [ ]:
#printing the table
database

In [ ]:
#saving the table to drive
database.to_csv('/content/drive/My Drive/database.csv', index = False) #drive path like /content/drive/My Drive/database.csv